# Synthetic Data - CS3110 Final Project

### Main Goal
Our team consists of Teddy Ruth, Joe Brennan, and Jordan Gottlieb. Our goal in this project is to go further in depth into the accuracies (or lackthereof) of synthetic data. How does accuracy compare across different marginals? What techniques can we use to maximize accuracy? What are some of the negative consequences of increasing accuracy?

TODO: 

1. Establish accuracy of 4-way marginal - done
2. Develop overlapping marginals - done
3. Establish accuracy of overlapping marginals - done
4. Run a series of queries on overlapping marginals
5. Run the same queries using laplace/gaussian mech on orignal data
6. Present findings

In [23]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

def pct_error_vec(orig, priv):
    errors = []
    for i in range(len(orig)):
        pct_err = np.abs(orig[i]-priv[i])/orig[i] * 100.0
        errors.append(pct_err)

    return errors


evs = pd.read_csv("Electric_Vehicle_Population_Data.csv")


## 1. Generating synthetic representations and data

### dp_marginal

The below function calculates the marginal of a given column and epsilon value. The function returns a dictionary where the keys are the column value, and the value is the chance of occurence over the whole dataset. For example, when passing in the `County` column, the function will return the counties in the dataset as key values, and the number of occurences over the whole dataset

In [2]:
def dp_marginal(col, epsilon):
    
    data = evs[col].value_counts()
    results = [x for x in data]
    noisy = [laplace_mech(v, 1, epsilon) for v in results]
    labels = evs[col].value_counts().index.to_list()
    syn_rep = {}
    
    for x in range(len(labels)):
        syn_rep[labels[x]] = max(0, noisy[x])
    
    total = sum(syn_rep.values())
    
    marginal = {}
    for x in labels:
        marginal[x] = syn_rep[x] / total
    return marginal


marginal = dp_marginal('County', 1.0)

### dp_synthetic_data_1way

This function generates synthetic data by generating a marginal for every column passed to the function and then combining data into a single dataframe to return. The function takes which columns, the number of rows to generate, and the epsilon value.

In [3]:
def dp_synthetic_data_1way(cols, n, epsilon):
    df_data = {}
    for col in cols:
        df_data[col] = []
        data = evs[col].value_counts().index.to_list()
        results = [x for x in data]
        marginal = list(dp_marginal(col, epsilon).values())
        synthetic = np.random.choice(results, size=n, p=marginal)
        
        for x in synthetic:
            df_data[col].append(x)
            
    dp_df = pd.DataFrame.from_dict(df_data)
    
    return dp_df

### syn_data1

Synthetic data of just the `County` column.

In [4]:
syn_data1 = dp_synthetic_data_1way(['County'], 100, 1.0)
print(syn_data1)

       County
0      Skagit
1    Thurston
2        King
3        King
4      Pierce
..        ...
95       King
96       King
97     Pierce
98       King
99  Snohomish

[100 rows x 1 columns]


### syn_data2
Synthetic data of `County` and `Make` by stitching together 2 1-way marginals

In [5]:
syn_data2 = dp_synthetic_data_1way(['County', 'Make'], 100, 1.0)
print(syn_data2)

       County       Make
0     Douglas      TESLA
1      Pierce      TESLA
2        King      TESLA
3        King       JEEP
4        King      TESLA
..        ...        ...
95       King       FORD
96       King     NISSAN
97       King       JEEP
98      Clark  CHEVROLET
99  Snohomish      TESLA

[100 rows x 2 columns]


The issue with `dp_synthetic_data_1way` is that it does not preserve any of the correlations in the dataset. It is merely taking 2 marginals and stitching them together. While this may generate good data if there is no preference to maintain correlations between columns in the data, this will not do a good job if it is desired to preserve these correlations. In order to address this we can generate a new set of synthetic data using 2 way marginals which preserves these correlations.

### dp_two_marginal

In [6]:
def dp_two_marginal(col1, col2, epsilon):
    hist = evs[[col1, col2]].value_counts()
    dp_hist = hist.apply(lambda x: laplace_mech(x, 1, epsilon))
    dp_hist = dp_hist.clip(lower=0)
    
    s = dp_hist.sum()
    
    dp_hist = dp_hist.to_frame().reset_index()
    dp_hist.rename(columns={0:'probability'}, inplace=True)
    
    for x in range(len(dp_hist)):
        val = dp_hist.at[x,'probability']
        dp_hist.at[x, 'probability'] = val / s
    
    
    return(dp_hist)

marginal2 = dp_two_marginal('County', 'Make', 1.0)

### dp_synthetic_data

Generates synthetic data of n rows when given a marginal

In [7]:
def dp_synthetic_data(n, marginal):
    samples = marginal.sample(n=n, replace=True, weights='probability')
    return samples

### syn_data3
Synthetic data generated using 2-way marginal of `County` and `Make`

In [8]:
syn_data3 = dp_synthetic_data(100, marginal2)
print(syn_data3)

      County        Make  probability
4       King   CHEVROLET     0.032403
78   Clallam       TESLA     0.001624
0       King       TESLA     0.258202
41  Thurston      NISSAN     0.003603
2       King      NISSAN     0.041887
..       ...         ...          ...
2       King      NISSAN     0.041887
10      King  VOLKSWAGEN     0.013963
45      King    POLESTAR     0.003260
8       King         KIA     0.018909
21   Whatcom       TESLA     0.007466

[100 rows x 3 columns]


### Note:
While we may not initially notice anything by constructing a 2 way marginal, what happens when we construct a 4 way marginal? Below we notice that the number of rows significantly increases. This is because with a 4 way marginal we group the data by 4 columns which leds to more specificity in grouping causing more and smaller groups. These small groups lead to a lower signal and more noise disruption when generating our marginal.

### dp_four_marginal

In [9]:
def dp_four_marginal(col1, col2, col3, col4, epsilon):
    hist = evs[[col1, col2, col3, col4]].value_counts()
    dp_hist = hist.apply(lambda x: laplace_mech(x, 1, epsilon))
    dp_hist = dp_hist.clip(lower=0)
    
    s = dp_hist.sum()
    
    dp_hist = dp_hist.to_frame().reset_index()
    dp_hist.rename(columns={0:'probability'}, inplace=True)
    
    for x in range(len(dp_hist)):
        val = dp_hist.at[x,'probability']
        dp_hist.at[x, 'probability'] = val / s
    
    
    return(dp_hist)

marginal4 = dp_four_marginal('County', 'Make', 'Electric Range', 'Model Year', 1.0)

### syn_data4
Synthetic data generated using 4-way marginal of `County`, `Make`, `Electric Range`, and `Model Year`.

In [10]:
syn_data4 = dp_synthetic_data(100, marginal4)
print(syn_data4)

         County        Make  Electric Range  Model Year  probability
4339      Lewis  VOLKSWAGEN               0        2021     0.000021
10         King       TESLA             291        2020     0.007742
20       Pierce       TESLA               0        2021     0.005086
276    Thurston         KIA               0        2022     0.000529
2047    Spokane         KIA              33        2023     0.000069
...         ...         ...             ...         ...          ...
2639   Franklin       TESLA             270        2019     0.000018
8     Snohomish       TESLA               0        2021     0.008598
0          King       TESLA               0        2023     0.078808
1029     Kitsap   CHEVROLET              35        2012     0.000124
30         King    CHRYSLER              32        2023     0.004019

[100 rows x 5 columns]


Our last method for generating accurate multidimensional marginals is by using overlapping marginals to preserve some of the correlations that we deem important while at the same time not utilizing high dimensional groupings that we would severly lower the signal of that data and hurt accuracy.

### dp_marginal_df and gen_sample

Below are two helper functions for our 'dp_synthetic_data_overlapping_marginals' function.

In [11]:
def dp_marginal_df(col, epsilon, df):
    f = lambda x: x + np.random.laplace(loc=0, scale=1/epsilon)
    hist_noisy = df[col].value_counts().apply(f)
     
    non_negative_syn_rep = np.clip(hist_noisy, 0, None)
    
    total = np.sum(non_negative_syn_rep)
    h = lambda x: x/total #normalized
    
    return non_negative_syn_rep.apply(h)

def gen_sample(marginal, cols, df):
    keys = df[cols].value_counts().keys()
    return np.random.choice(keys, size=1, p=marginal)

### dp_synthetic_data_overlapping_marginals

In [12]:
def dp_synthetic_data_overlapping_marginals(n, epsilon):
    synth_df = pd.DataFrame(columns=['County','Make','Electric Range','Model Year'])
    
    for x in range(n):
        # generate marginal of County in evs
        m_County = dp_marginal_df('County', epsilon, evs)
        # get single synthetic County using marginal and evs
        County_syn = gen_sample(m_County, 'County', evs)
        
        # filter evs to only contain people with that County
        filtered_County = evs[evs['County']==County_syn[0]]
        
        # generate marginal of Make in filtered dataframe
        m_Make = dp_marginal_df('Make', epsilon, filtered_County)
        # get single synthetic Make using marginal and filtered evs
        Make_syn = gen_sample(m_Make, 'Make', filtered_County)
        
        # filter evs to only contain people with that Make
        filtered_Make = evs[evs['Make']==Make_syn[0]]
        
        #repeat
        m_Range = dp_marginal_df('Electric Range', epsilon, filtered_Make)
        Range_syn = gen_sample(m_Range, 'Electric Range', filtered_Make)
        
        filtered_Range = evs[evs['Electric Range']==Range_syn[0]]
        
        m_Year = dp_marginal_df('Model Year', epsilon, filtered_Range)
        Year_syn = gen_sample(m_Year, 'Model Year', filtered_Range)
    
        synth_df.loc[x] = [County_syn[0], Make_syn[0], Range_syn[0], Year_syn[0]]
    return synth_df

syn_data = dp_synthetic_data_overlapping_marginals(2000, 1.0)

## 2. Evaluation  of percent errors of synthetic data vs. original data

### pct_error_1Dmarginal

In [13]:
def pct_error_1Dmarginal(marginal):

    def gen_samples(cols, n, epsilon):
        df_data = {}
        for col in cols:
            df_data[col] = []
            data = evs[col].value_counts().index.to_list()
            results = [x for x in data]
            marginal = list(dp_marginal(col, epsilon).values())
            synthetic = np.random.choice(results, size=n, p=marginal)
            
            for x in synthetic:
                df_data[col].append(x)
                
        dp_df = pd.DataFrame.from_dict(df_data)

        return dp_df
    
    syn_data = gen_samples([column], len(evs), 1.0).value_counts()
    ev_makes = evs[column].value_counts()


    errors = pct_error_vec(list(syn_data), list(ev_makes))

    errors_sum = sum(errors)

    return((errors_sum / len(errors)))


column = 'Make'
marginal = dp_marginal(column, 1.0)
mean_error = pct_error_1Dmarginal(marginal)
print(f"Mean Error for Column '{column}': " + str(mean_error))


Mean Error for Column 'Make': 8.064299455095451


In the above cell we determine the mean percent error of a single iteration of generating a single column of synthetic data. This information while useful is still volatile, below I will establish the mean percent error over 50 iterations.

In [14]:
def mean_pct_error_1Dmarginal(column):
    errors = []

    for x in range(50):
        marginal = dp_marginal(column, 1.0)
        error = pct_error_1Dmarginal(marginal)
        errors.append(error)

    mean_error = sum(errors) / len(errors)

    return (mean_error)

mean_pct_error_1Dmarginal('Make')

7.580059013824462

Running this code over 50 iterations yields us an average percent error of ~7-8% variance from the initial dataset. While this number is not incredibly high, it does not indicate possible errors over multiple columns. In the next cells we will determine the percent error over multiple columns. We will start with the percent error of the previous 4 way marginal

### pct_error_4Dmarginal
The function below generates the percent error of a 4D marginal generated synthetic dataset by comparing each of its column values to the column values of the original dataset.

In [15]:
def pct_error_4Dmarginal(col1, col2, col3, col4):
    fourWay = dp_four_marginal(col1, col2, col3, col4, 1.0)
    error_col1 = pct_error_1Dmarginal(fourWay[col1])
    error_col2 = pct_error_1Dmarginal(fourWay[col2])
    error_col3 = pct_error_1Dmarginal(fourWay[col3])
    error_col4 = pct_error_1Dmarginal(fourWay[col4])

    return error_col1, error_col2, error_col3, error_col4
    
pct_error_4Dmarginal('County', 'Make', 'Electric Range', 'Model Year')

(8.665564214849736, 6.094136499559047, 6.617108376818698, 6.353867610809675)

In order to obtain a more accurate result we generate 50 percent errors for each of the columns and then take the mean of each.

In [16]:
def mean_pct_error_4Dmarginal(col1, col2, col3, col4):
    errors1 = []
    errors2 = []
    errors3 = []
    errors4 = []

    for x in range(50):
        error1, error2, error3, error4 = pct_error_4Dmarginal(col1, col2, col3, col4)
        errors1.append(error1)
        errors2.append(error2)
        errors3.append(error3)
        errors4.append(error4)

    mean_error1 = sum(errors1) / len(errors1)
    mean_error2 = sum(errors2) / len(errors2)
    mean_error3 = sum(errors3) / len(errors3)
    mean_error4 = sum(errors4) / len(errors4)
    
    return (mean_error1, mean_error2, mean_error3, mean_error4)

mean_pct_error_4Dmarginal('County', 'Make', 'Electric Range', 'Model Year')

(7.044318630492233, 8.968498732089751, 7.20537490880694, 6.462831046266944)

### pct_error_overlapping_marginals

In [17]:
def pct_error_overlapping_marginals(col1, col2, col3, col4):
    marginal = dp_synthetic_data_overlapping_marginals(2000,1)
    error_col1 = pct_error_1Dmarginal(marginal[col1])
    error_col2 = pct_error_1Dmarginal(marginal[col2])
    error_col3 = pct_error_1Dmarginal(marginal[col3])
    error_col4 = pct_error_1Dmarginal(marginal[col4])

    return error_col1, error_col2, error_col3, error_col4
    
pct_error_overlapping_marginals('County', 'Make', 'Electric Range', 'Model Year')

(12.71354527335612, 4.634052696226412, 12.416822780717728, 9.63149396684025)

It is possible to evaluate this multiple times and take the average; however, due to the nature of the task it takes a significant amount of time (10 - 15 minutes). We have provided the code to run the error of overlapping marginals multiple times for those interested.

In [24]:
def mean_pct_error_overlapping_marginal(col1, col2, col3, col4):
    errors1 = []
    errors2 = []
    errors3 = []
    errors4 = []

    for x in range(3):
        error1, error2, error3, error4 = pct_error_overlapping_marginals(col1, col2, col3, col4)
        errors1.append(error1)
        errors2.append(error2)
        errors3.append(error3)
        errors4.append(error4)

    mean_error1 = sum(errors1) / len(errors1)
    mean_error2 = sum(errors2) / len(errors2)
    mean_error3 = sum(errors3) / len(errors3)
    mean_error4 = sum(errors4) / len(errors4)
    
    return (mean_error1, mean_error2, mean_error3, mean_error4)

mean_pct_error_overlapping_marginal('County', 'Make', 'Electric Range', 'Model Year')

### Queries

We built these queries to demonstrate the percent errors between the laplace mechanism on the original dataset and the synthetic data set. 

In [20]:
""" COUNT QUERIES """
def count_query(df, col, equals):
    return len(df[df[col]==equals])


evs = evs[0:len(syn_data)]
syn_count = count_query(syn_data, 'Make', 'TESLA')
lp_count = laplace_mech(count_query(evs[0:len(syn_data)], 'Make', 'TESLA'), 1, 1.0)
count = len(evs[evs['Make'] == 'TESLA'])


syn_count_err = pct_error(syn_count, count)
lp_count_err = pct_error(lp_count, count)




""" MEAN QUERIES """


def mean_query(df, col):
    df_sum = df[col].sum()
    df_len = len(df)
    return df_sum / df_len


def dp_mean_query(df, col, epsilon):
    df_sum = laplace_mech(df[col].sum(), 400, epsilon / 2)
    df_len = laplace_mech(len(df), 1, epsilon / 2)
    return df_sum / df_len




syn_mean = mean_query(syn_data, 'Electric Range')
mech_mean = dp_mean_query(evs, 'Electric Range', 1.0)
ev_mean = mean_query(evs, 'Electric Range')


syn_mean_err = pct_error(syn_mean, ev_mean)
mech_mean_err = pct_error(mech_mean, ev_mean)


print("SYNTHETIC")
print(syn_count_err)
print(syn_mean_err)

print("\n\nMECHANISMS")
print(lp_count_err)
print(mech_mean_err)


SYNTHETIC
4.385026737967915
2.5218234723569313


MECHANISMS
0.30336200709379274
0.460952400547485
